In [1]:
!pip install transformers -q
!pip install wandb -q
! pip install transformers
! pip install datasets
! pip install sentencepiece
! pip install rouge_score
! pip install py7zr
# Code for TPU packages install
# !curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 708.8 kB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Importing stock libraries
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

# WandB – Import the wandb library
import wandb

In [3]:
# Checking out the GPU we have access to. This is output is from the google colab version. 
!nvidia-smi

Sat Apr 22 01:25:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Preparing for TPU usage
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

In [5]:
# Login to wandb to log the model run and all the parameters
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
from huggingface_hub import login
login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage for finetuning the model and to prepare it for predictions

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [8]:
# Creating the training function. This will be called in the main function. It is run depending on the epoch value.
# The model is put into train mode and then we wnumerate over the training loader and passed to the defined network 

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
        loss = outputs[0]
        
        if _%10 == 0:
            wandb.log({"Training Loss": loss.item()})

        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [9]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [10]:
# WandB – Initialize a new run
wandb.init(project="T5_finetune_dialogue")

# WandB – Config is a variable that holds and saves hyperparameters and inputs
# Defining some key variables that will be used later on in the training  
config = wandb.config          # Initialize config
config.TRAIN_BATCH_SIZE = 8    # input batch size for training (default: 64)
config.VALID_BATCH_SIZE = 4   # input batch size for testing (default: 1000)
config.TRAIN_EPOCHS = 2        # number of epochs to train (default: 10)
config.VAL_EPOCHS = 1 
config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
config.SEED = 42               # random seed (default: 42)
config.MAX_LEN = 512
config.SUMMARY_LEN = 50#150 

# Set random seeds and deterministic pytorch for reproducibility
torch.manual_seed(config.SEED) # pytorch random seed
np.random.seed(config.SEED) # numpy random seed
torch.backends.cudnn.deterministic = True

# tokenzier for encoding the text
tokenizer = T5Tokenizer.from_pretrained("t5-base")

wandb: Currently logged in as: jainr3 (rahul96jain). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [11]:
# Importing and Pre-Processing the domain data
# Selecting the needed columns only. 
# Adding the summarzie text in front of the text. This is to format the dataset similar to how T5 model was trained for summarization task. 
#df = pd.read_csv('./data/news_summary.csv',encoding='latin-1')
#df = df[['text','ctext']]
#df.ctext = 'summarize: ' + df.ctext
#print(df.head())
import datasets
from datasets import load_dataset

def merge_splits(data):
  data = datasets.concatenate_datasets([data["train"], data["validation"], data["test"]])
  return data

data_AMI = merge_splits(load_dataset("knkarthick/AMI"))
data_samsum = merge_splits(load_dataset("samsum"))
data_dialogsum = merge_splits(load_dataset("knkarthick/dialogsum"))

def flatten(example):
    return {
        "ctext": "summarize: " + example["dialogue"],
        "text": example["summary"],
    }


data_AMI = data_AMI.map(flatten, remove_columns=["id"])
data_samsum = data_samsum.map(flatten, remove_columns=["id"])
data_dialogsum = data_dialogsum.map(flatten, remove_columns=["id", "topic"])

dataset = datasets.concatenate_datasets([data_AMI, data_samsum, data_dialogsum])
dataset = dataset.remove_columns(["dialogue", "summary"])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--AMI-e0c25640a9a75f54/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


...

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/279 [00:00<?, ? examples/s]

Map:   0%|          | 0/16369 [00:00<?, ? examples/s]

Map:   0%|          | 0/14460 [00:00<?, ? examples/s]

In [12]:
dataset

Dataset({
    features: ['ctext', 'text'],
    num_rows: 31108
})

In [13]:
# Creation of Dataset and Dataloader
# Defining the train size. So 80% of the data will be used for training and the rest will be used for validation. 
#train_size = 0.8
#train_dataset=df.sample(frac=train_size,random_state = config.SEED)
#val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
#train_dataset = train_dataset.reset_index(drop=True)

train_data_txt, validation_data_txt = dataset.train_test_split(test_size=0.2).values()
train_dataset = pd.DataFrame(train_data_txt)
val_dataset = pd.DataFrame(validation_data_txt)

print("FULL Dataset: {}".format(dataset.num_rows))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(val_dataset.shape))


# Creating the Training and Validation dataset for further creation of Dataloader
training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

# Defining the parameters for creation of dataloaders
train_params = {
    'batch_size': config.TRAIN_BATCH_SIZE,
    'shuffle': True,
    'num_workers': 0
    }

val_params = {
    'batch_size': config.VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
    }

# Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)

FULL Dataset: 31108
TRAIN Dataset: (24886, 2)
TEST Dataset: (6222, 2)


In [14]:
# Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
# Further this model is sent to device (GPU/TPU) for using the hardware.
model = T5ForConditionalGeneration.from_pretrained("t5-base")
model = model.to(device)

# Defining the optimizer that will be used to tune the weights of the network in the training session. 
optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)


In [15]:
# Log metrics with wandb
wandb.watch(model, log="all")

[]

In [16]:
# Training loop
print('Initiating Fine-Tuning for the model on our dataset')

for epoch in range(config.TRAIN_EPOCHS):
    train(epoch, tokenizer, model, device, training_loader, optimizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Initiating Fine-Tuning for the model on our dataset


/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Loss:  5.504509449005127
Epoch: 0, Loss:  1.4463902711868286


KeyboardInterrupt: ignored

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
val_dataset[:1]

,ctext,text
0,summarize: Ashley: Hey who down for Yoga? \r\n...,"Laurie, Ashley, Matt and Jesse are going for s..."


In [19]:
partial_val_set = CustomDataset(val_dataset[:50], tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
partial_val_loader = DataLoader(partial_val_set, **val_params)

In [20]:
# Validation loop and saving the resulting file with predictions and acutals in a dataframe.
# Saving the dataframe as predictions.csv
print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
for epoch in range(config.VAL_EPOCHS):
    #predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    predictions, actuals = validate(epoch, tokenizer, model, device, partial_val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv('/content/drive/MyDrive/predictions.csv')
    print('Output Files generated for review')

Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe


/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Completed 0


/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Output Files generated for review


In [21]:
torch.save(model.state_dict(), "/content/drive/MyDrive/t5-model-v3.pth")

In [22]:
# saving to huggingface
# https://huggingface.co/transformers/v4.0.1/model_sharing.html#:~:text=In%20order%20to%20upload%20a,huggingface.co%2Fnew%3E%20.
#!transformers-cli login
#!huggingface-cli repo create t5-finetuned-meetings
!git clone https://huggingface.co/jainr3/t5-finetuned-meetings
!git lfs install

Cloning into 't5-finetuned-meetings'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 17 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (17/17), 3.26 KiB | 833.00 KiB/s, done.
Filtering content: 100% (2/2), 851.15 MiB | 49.96 MiB/s, done.
Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.


In [23]:
model.save_pretrained("t5-finetuned-meetings")
tokenizer.save_pretrained("t5-finetuned-meetings")

('t5-finetuned-meetings/tokenizer_config.json',
 't5-finetuned-meetings/special_tokens_map.json',
 't5-finetuned-meetings/spiece.model',
 't5-finetuned-meetings/added_tokens.json')

In [24]:
%cd /content/t5-finetuned-meetings
!git add --all
!git status
%cd /content

/content/t5-finetuned-meetings
On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   pytorch_model.bin

/content


In [25]:
!git config --global user.email "Rahul96Jain@gmail.com"
!git config --global user.name "Rahul Jain"

In [27]:
%cd /content/t5-finetuned-meetings
!git commit -m "Second version of the t5 finetuned meetings model and tokenizer."
%cd /content

/content/t5-finetuned-meetings
[main 9b5789c] Second version of the t5 finetuned meetings model and tokenizer.
 1 file changed, 1 insertion(+), 1 deletion(-)
/content


In [29]:
%cd /content/t5-finetuned-meetings
!git push
%cd /content

/content/t5-finetuned-meetings
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 408 bytes | 408.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
To https://huggingface.co/jainr3/t5-finetuned-meetings
   1421fda..9b5789c  main -> main
/content


## Running inference on my dialogue

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")
model = model.to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/t5-model-v3.pth"))
tokenizer = T5Tokenizer.from_pretrained("t5-base")

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [32]:
my_input_text = " Hi, this is Eric Johnson. It's February 18, 2021, and this is the engineering key review at GitLab. So I've got number four in the agenda, which is a proposal to break up this meeting into four department key reviews. So currently this is engineering. Development quality, security, and UX infrastructure and support to their own key reviews already. I have the reasons why increased visibility able to go deeper. Increased the objectivity with which my reports can manage their groups, allow me more time to focus on new markets and allow me to shift into more of a question asker mode, then generating content and answering questions in these meetings. And to avoid adding three net new meetings to stakeholders counters, I propose we do a sort of two month rotation. So month one development quality go month two security and UX would go. How do people feel about that proposal? I think in the group conversations it's working really well. So I'm supportive and this is the smallest thing. Maybe we need four meetings a month, like it's the biggest department, it's super central. But you proposed this. I don't. I could see either way. So let's stick with the proposal. Go, we'll try it and we'll we can be flexible. I mean, development is larger. Maybe they go more frequently or something. I'll see how it goes. All right. And then I've got number five, which is we've got R&D overall MR rate. And we also have R&D wider MR rate, both as top level KPIs for engineering. So the difference between them in the simplest sense is that R&D wider MR rate includes both community contributions and community MRs. The problems I see with this are that one, the wider MR rate, the one that includes internally and external MRs, it duplicates the overall MR rate, which is. The wider MR rate should just be external, right? And then overall should be narrow plus wider. Oh, yeah. Like we say the wider community. Right, right, right. Okay. So there's. I have to check the taxonomy. Well, like, can you can you confirm? That's that's that's, that's, that's reasoning is my understanding as well. Yeah, I believe water MR rate just captures community contributions. Only and no internal. Yeah, and the reason we measure data is that like one of the most likely failure modes is that we lose the community. Yeah. So we're get, or we're gets goofy is the. When you look at a specific team within the company, there could be contributions outside of that that aren't community contributions. They would be viewed community contributions by that group, but effectively they're not from outside the company. So that's why we use wider to kind of reflect that. And narrow is very specific to the team. But are you saying that if someone in plan contributes to verify its viewed as wider? Not quite that plan plan and verify are just fine. It's when you look at like the development versus infrastructure infrastructure will oftentimes contribute to developments work, but it won't be counted as MRs. Yeah, that's a, that's a potential bit of funkiness that we should talk about separately. I didn't have that in my sort of critique of this, but that that doesn't necessarily make intuitive sense to me. So, so then I think part of my critique of this can be thrown out because it's not as duplicative as I thought, but I still think there's a problem with R&D wider MR rate, which is this thing doesn't really move. In part because it's a rate, so it feels like the way to drive this up is to specifically drive community authors to contribute more than one MR per month. That's how this moves up because it's a productivity rate like we use internally. And that doesn't necessarily feel like the right thing because there's scenarios in which this goes up. We've actually got less contributions overall and less contributors overall. Wait, wait, wait a second. So you're saying that R&D wider MR rate is number MRs per external contributor. Oh my goodness, that should not be the thing. It should be contributions for GitLab team member. So to contribute those, the thing above the division is the external ones. The thing below is the number of team members at GitLab. Is that the case, Lily? I'm checking right now. I think so in our MRs per team member. So unless we start calling people outside the company team member, then it shouldn't be that. Yeah, just clarifying here. So our numerator is community contributions and then our denominator is GitLab team members. So it's not per external member. So we've got what we're doing there, Eric, is we're not trying to say how many MRs does someone send if they send something? How many MRs from external do we get for the size of our organization? Sorry, I have a childhood emergency outside the door. So maybe explain the context behind this. The context is as we grow as a company, we should make sure we keep the community up. Like the logical thing is for the community to flatline and the size of the org to go and before you know, you've kind of outgrown the wider community. Yeah, what I'm seeing is we created this pre-sophisticated taxonomy with prefixes and postfixes to talk about these things. But in reality, we've only got two of them. And it's we keep forgetting, we have a hard time discussing this thing. So I'd rather just name them simply two names for what they are rather than using the taxonomy. But also like in F, I have this proposal of like, what if we just track as a KPI of the percentage of total MRs that come from the community over time? And we would see that drop. I love that. Let's do that instead. But the thing, the thing why we have this complex thing is because you can game that you want to game that you just produce fewer MRs with the engineers that get one. So if you drive that really hard and say this is your number one goal, it's very easy to achieve is just tell all your engineers to produce half. Yeah, so we have we have different metrics to prevent that from happening the same way that like support SLAs and asset kind of buttress one another. I think we're we're robust to that. But simplifying this would make these conversations. If you as our CTO don't even understand them, we went overboard. So I'm supportive. And I forgot and I was agreeing this stuff this morning and like, there's a problem with this and then you just remind me of the context. So yeah, I can't really have my head. A percentage that come from the community. I love that. It's what all our investors ask about. Let's do that. Okay. Cool. So, um, Lily, if you can work with Max to make that transition of the great. And I'll hold the one that we're talking about. That's from the mulberry. Thanks. I'm on the call. Sorry, it was a bit late. Timeline. We do have PIs on the raw number of community MRs and we can make the shift. And why they're confirming from the definition, I think why they only counts for community. And that's that's what the definition is. Cool. All right. So number six, then Christopher. Sorry, I was looking up to see if I had the percentage graph because I think we played around with this at one point and had a draft of that. Probably about five months back, if I can remember, Lily. Just in FYI month of February, if you were looking at any particular metrics, particularly in development at MR rate. We haven't had updates in four days. There's apparently a lag issue that's been problematic for the data team to basically get updated metrics and they're working on that. Okay. I'm sorry, Max. You get the next one. So yeah, there's some some color there on the on the medication, the lag later on. On to on to seven. We continue. I said FYI in addition to the TBI status. I'm sorry. I wanted to just touch on the postgres replication issue there real quick. I've been trying to get my arms wrapped around it. Um, do we have the right attention to this? This is kind of Eric. I don't know if you were commenting on. Hitting towards this in the last meeting around some of the infrastructure improvements on the product side. I'm just not quite sure whose responsibility is to focus on getting a handle on some of the constraints we have on replication. So what I was mentioning in the in the product queue review about an hour ago is I think is sort of like unrelated. And so I think the DRI needs to be your kind of data engineering team. But of course there's a dependency on infrastructure because that's where the data is being piqued from. They they do on that data source. So let's say for the replication lag on that slave host where I'm sorry not the on the the secondary host where the data is being pulled from like infrastructure would be the DRI for that. And so any escalations but well all those. And I know we have an action plan for that as far as creating another dedicated host just for the data team to pull from. I did ask I saw that issue and I did talk to Craig gums a little bit as well on the database side just to see if there's some database improvements and I'm still trying to figure out, you know, if it's truly just dedicated computation old sort of resource a server. Or if there's actually some some database tuning that needs to occur, do you have a sense of that. There's so I'd say it's it's three different things. It's having a dedicated host that doesn't have conflicting query traffic coming from other other workloads. There are some tuning performance or tuning improvements to be made. And then there's also improvements in and this is where it does maybe relate a little bit to what the topic was in the last review basically the overall demand on the database layer from from dot com activities and like improving those. It's definitely not just one of those things that one of the most specific actions we're going to take though is separating out and having a dedicated host so that we're just dealing with the profile of the data engineering traffic on there. And not having conflicting query. And conflicting queries affect the ability to update the replication. Steve, I definitely want to partner with you on this one because I think the the demand on those databases is only going to increase. It's not going down. And I think we need to get. I'm still unclear on where to focus and to get the biggest bang for the buck. I think of the computational resource dedication that's going to be a good thing, but I'll probably going to squeeze the balloon. Next area will will honor itself. Okay, I'll put into the infricky review for next week. Okay, on this issue. Thank you, Steve. So then back to mecon 7 or. Yes. Thank you. And Rob was in these assassins that this morning as well. We have the attention there. Number seven, just provide provide update on previous conversations. We continue to improve defect tracking and against SLOs. There is a first iteration P.I that we are experimenting to show percentage of defects meeting. And we are going to be able to see the SLOs key findings as ones are hovering at 80% as to at 60. We've been focused mostly on S1 as to this point as hence why S3 and S4s are a lower. And this will likely be the case. We are also in point B are working on the measurement for average open box age. This would give us a whole picture of what's what's left. So the age goes up and down. We are cleaning the backlog. The average age should go down as well. There's no P.I yet, but I just want to update and show up beyond this. It's not a off track. Number C Craig on S2. Yeah, just looking through the charts. And I noted that there's a spike in meantime to close. Yeah, and just want to see if you had any insight into that for us. This is the S2. Yeah, this is where the point B on age and supplemental charts in the back end helps. So I haven't seen a dip in age nor the count overall. I think it's the latter. We need to dig in a bit deeper in that. And also the data lag. I would like to re-evaluate when we have a whole picture of when everything is synced in as well. Christie, you have some insights. Yeah, I'm just wondering if part of this could be the fact that we changed the severity across the board for MRs to S2. And so we may have some older bugs in there that hadn't been addressed because they were at a lower severity. Now we've moved them to S2 and maybe that caused a little spike. That could be the case that we did in a limited fashion. It won't be a huge volume. We also iterated after that to pin on priority since product owns prioritization. So I wouldn't account it entirely to that. I mean, this isn't the infertile key review, but. I know that they've gotten backed up on those issues. So if some some good portion of those are infertile created or related, then that may be lifting it as well. I can take the deeper dig in and then provide an update next time. I think we need extra debug slicing of the data here. So you would like to go to point eight. Yeah, we are now measuring S1 as to SLO achievement with closed bugs. But if you then look at the number of bugs, it's exponential growth. And then it will be trivial to us to achieve 100% SLO achievement. If you just look at close bugs, even though there would be a major problem in the company, 99% of all bugs are overdue. As long as I only close ones that are still within the SLO, I'll have great achievement. So I think we shouldn't be looking at the closed bugs. I think we should be looking at open bugs. The entire population or percentage of those is within the SLO time. I think we're doing it the wrong way. Thanks for the thanks for the feedbacks that. Hence why we wanted to have the average age to measure what's outside in the open. And then make this iteration to measure also measure focus on the age of all opened, including open bugs. This is also something we have discussed with Christopher in the next iteration as well. And we happy when happy to adjust. So the key. Yes, go ahead. So average age would get closer to it. It's not what I'm proposing. What I'm proposing is off the open bugs are percentages outside of SLO. So display it as a percentage. You do now just do it about the open bugs month or close. Got it. Okay. The exceeding SLO for open bugs. Yeah, or open bugs that are within SLO. So you have a chart that should go up and to the right like everything else. Sounds great. We can take it to the next data metrics work stream to deliver this. Cool. Thanks. Is it pretty good? Yeah, I like it. You make you have to figure out how to. Because we like to be able to have charts that we can historically reconstruct if we need to. So in tickets close out, you need to go through their history to figure out at this time when it was open. Did it breach the SLO or not? That's a good point. This might be much harder computationally. And so I totally respect if we can do it for that reason. Yeah, I think that's a good point. Yeah, I think that's a good point. Cool. Nine. Yeah, I just wanted to ask the team like I went through all the key meeting metrics. Everything looked in line with prior periods and look good. Is there anything the team wants to call out? It's mostly that we should be watching. Yeah, I'll call out. So the good news is in Q4. We had our smallest decline over several quarters. So we only went down by a tenth of a point. The quarter previous was point six or six tenths of a point and the quarter before that was a full point. So. We see this as an improvement, even though it was still a decline, but it's still a decline. Obviously, we want this actually tracking in an upward direction. We also don't have enough data to know whether or not this is an actual real trend up. So I'm optimistic. I think this is a good thing. We have had a much keener focus on us over the past several quarters. And that's why I think, okay, the work that we've done. I think actually is catching up and getting noticed in sus, but we got to keep an eye on it. We can't. We cannot assume that that's the case. Yeah, and the bug discussion above just kind of points out that like we have an underlying problem right now in our metrics measurement. So if we change the measurements to reflect that, then hopefully we're in good shape. If we don't and we flat line and address it so that we flat line open S1s and S2s, you will see a temporary jump in and above SLOs. As we clear out that back, I'll go over that period of time. So we have a point C, which is similar to infrastructure. We need to get more security work prioritized or hearing that from the team, but neither that problem or that activity is sort of currently reflected in our security metrics. So we have some work to do long term to make sure the we see things like that in the metrics and the measurements that we're making. So back to you said 10. So now that Nero MR rate seems significantly below target. And maybe I helped that it would bounce back from December. I think it bounced back, but not back on target. Any any context there? What's going on? Yeah, so with family and friends days, we actually had some heavier of vacation days. And we historically have one thing to know is that we are actually at a higher MR rate. If you look, if you go back the last 18 months, we're actually at a higher Nero MR rate than we were back in each month this year. So if you compare October to October, November to November and January, I'm sorry, October, November, December and January, comparatively to last year, which will find is we're between a half point and a 1.5. MR rate above where we were in the month of previous year. That's great contact. Thank you, Chris, for good work. Yeah, so the expectation is is that February is a short month. We were at I think 16 work days with friends and family day and other things. Obviously seven carriages in Texas doesn't help things either for the folks who are working in Texas. But hopefully the rest of the team is being effective. I was hoping to see a better result right now, but with four or five days, particularly around release week. That's usually when we see do do see a little bit higher activity. So that's not accounted for yet. But you know, Marches, Marches when I'm expecting kind of see a real rebound much like we did last year. Awesome. Thanks. The other context I'll give is we now do time series targets. So when we change the target, you'll see they're reflected in the line. So if we were to look back historically here, the goal here was actually lower and Christopher was ambitious. So we kept raising it. We kept meeting that. So it should stare step here and we could go back and reconstruct that if we really wanted to. And then ignoring the sort of the seasonal dip here, we raised it to I think 11. And then we realize we're kind of hitting that point of doing and raising returns and the right thing to do business wise. And this is in our FY 22 direction is hold the line at productivity. Let's start to raise other things related to quality security, availability and what not. So that's kind of what you are, you're seeing this bump is we raised it and we realized, okay, that's not the we shouldn't raise it anymore. And we brought it back down to 10. So 10 will be the dark going forward. I'm going to try to get better a lot of other things while preventing this from dipping. And just want to call out that it's not necessary. Like, and higher MR rates should also help to address security and quality and other things because you're more productive. So you can fix more things so it's not necessarily opposite. But I agree with that. Let's hold the line 10 is then is a great number and and focus on other indicators to improve that makes it on a sense. Cool. Well said. All right, that's the agenda. Anyone want to vocalize anything else? Great. Well, thanks everybody. And I'm going to go check on my four year old and see if she got what whatever she needed. So cheers and toxic."

In [33]:
val_params = {
    'batch_size': config.VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
    }

In [37]:
my_data = pd.DataFrame(columns=['ctext', 'text'])

def get_input_sentences(start_idx):
  # Get whole sentences up to X number of characters; don't break sentences up
  end_idx = start_idx + 2048
  text = my_input_text[start_idx:end_idx]
  end_sentence_chars = [".", "?", "!"]
  next_start_idx = end_idx
  while text[-1] not in end_sentence_chars and end_idx < len(my_input_text):
      text = text[:-1]
      next_start_idx -= 1
  #print(len(tokenizer.tokenize(text)))
  return text, next_start_idx

start = 0
input_text = []
summarized_text = []
while start < len(my_input_text):
  input, start = get_input_sentences(start)
  input_text.append(input)
  print("input text \n" + input)

for i, input in enumerate(input_text):
  my_data.loc[i] = ["summarize: " + input, input]

#my_data.loc[0] = ["summarize: " + my_input_text, ""]

my_data = CustomDataset(my_data, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
my_data_loader = DataLoader(my_data, **val_params)

input text 
 Hi, this is Eric Johnson. It's February 18, 2021, and this is the engineering key review at GitLab. So I've got number four in the agenda, which is a proposal to break up this meeting into four department key reviews. So currently this is engineering. Development quality, security, and UX infrastructure and support to their own key reviews already. I have the reasons why increased visibility able to go deeper. Increased the objectivity with which my reports can manage their groups, allow me more time to focus on new markets and allow me to shift into more of a question asker mode, then generating content and answering questions in these meetings. And to avoid adding three net new meetings to stakeholders counters, I propose we do a sort of two month rotation. So month one development quality go month two security and UX would go. How do people feel about that proposal? I think in the group conversations it's working really well. So I'm supportive and this is the smallest t

In [38]:
print('Now generating summaries on our fine tuned model for my dataset and saving it in a dataframe')
for epoch in range(config.VAL_EPOCHS):
    predictions, actuals = validate(epoch, tokenizer, model, device, my_data_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv('/content/drive/MyDrive/my_predictions-v3.csv')
    print('Output Files generated for review')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Now generating summaries on our fine tuned model for my dataset and saving it in a dataframe


/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Completed 0
Output Files generated for review


In [39]:
final_df

,Generated Text,Actual Text
0,Eric Johnson proposes to break up the engineer...,"Hi, this is Eric Johnson. It's February 18, 20..."
1,water MR rate just captures community contribu...,"That's that's that's, that's, that's reasoning..."
2,Lily and Eric are looking at the number of tea...,"So to contribute those, the thing above the di..."
3,Lily and Christopher are talking about a perce...,But simplifying this would make these conversa...
4,he thinks the DRI needs to focus on getting a ...,I'm just not quite sure whose responsibility i...
5,Steve wants to partner with him on this one be...,"Steve, I definitely want to partner with you o..."
6,the severity across the board for MRs has chan...,"Yeah, I'm just wondering if part of this could..."
7,the team is optimistic. They have had a much k...,So display it as a percentage. You do now just...
8,Chris and the team are in good shape. They nee...,So if we change the measurements to reflect th...
9,Marches is expecting a real rebound much like ...,"But you know, Marches, Marches when I'm expect..."


In [40]:
output_summary = ' '.join(final_df["Generated Text"].astype(str))

In [41]:
output_summary

"Eric Johnson proposes to break up the engineering key review into four department key reviews. Then he proposes a sort of two month rotation, so development quality go month two security and UX would go. water MR rate just captures community contributions. Only and no internal contributors. Lily and Eric are looking at the number of team members at GitLab. They're not talking about how many MRs they get for the size of their organization. Eric wants to make sure we keep the community up, but Eric prefers to use the taxonomy instead. Lily and Christopher are talking about a percentage that comes from the community. They'll make the transition of the great. he thinks the DRI needs to focus on getting a handle on some of the constraints we have on replication. I think there's a dependency on infrastructure because that's where the data is being piqued from. Craig gums also talked about creating another dedicated host just for the data team to pull from. Steve wants to partner with him on